# Objects

Objects are a bunch of data with some attitude. Individuals like you and me are objects. We are data: our name, age, gender, credit score, income, traffic tickets, social security number, and dark secrets like a preference for pineapple on pizza.

Our behavior is our attitude. It determines how we share the data we are, with others. We readily answer the question "what's your name" but we think twice before revealing our social security number.